In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import random
import math
import timeit
import xml.etree.ElementTree as ET
import os
from bs4 import BeautifulSoup


In [2]:
path = "C:\\Users\\Raaed\\Documents\\SEM 5\\Information Retrieval Assignment\\Assignment 2\\corpus\\corpus\\fulltext"

In [3]:
total_doc = []
i = 0
for file in os.listdir(path):
    i = i+1
    if(i>400):
        break
    with open(os.path.join(path,file),'r') as f:
        data = f.read()
        bs_data = BeautifulSoup(data,"xml")
        b_catch = bs_data.find_all('catchphrase')
        doc = ""
        for c in b_catch:
            phrase = c.text
            k=phrase.split(">")
            doc= doc + k[1] + " "
        total_doc.append(doc)

In [4]:
tokenizer = RegexpTokenizer("[\w']+")
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer() 


In [5]:
for i in range(len(total_doc)):
    tokens = tokenizer.tokenize(total_doc[i])
    stopset = set(stopwords.words('english'))
    filtered_sentence = [w for w in tokens if not w in stopset]
    stemmed_sentence = [stemmer.stem(w) for w in filtered_sentence]
    lemmatized_sentence = [lemmatizer.lemmatize(w) for w in stemmed_sentence]
    total_doc[i] = lemmatized_sentence

In [6]:
k_shingles = 2
shingle_set = {}

In [7]:
# Constructing the K shingle_set
##count=0
for i in range(len(total_doc)):
    for j in range(len(total_doc[i])- k_shingles):
        shingle = total_doc[i][j:j+k_shingles]
        shingle = ' '.join(shingle)
        if shingle not in shingle_set.keys():
            a = np.zeros(len(total_doc))
            a[i] = 1
            shingle_set[shingle] = a
        else:
            shingle_set[shingle][i] = 1
            ##count=1

In [8]:
i=0 ##shingle_id mapped to shingles
shingleID = [i for i in range(len(shingle_set))]
shingleMap = []
for key in shingle_set.keys():
    shingleMap.append([i,key])
    i+=1

In [9]:
N_HASHES = 10
hash_funcs = []
for i in range(N_HASHES):
    perm = random.sample(shingleID, len(shingleID))
    while perm in hash_funcs:
        perm = random.sample(shingleID, len(shingleID))
    hash_funcs.append(perm)

In [10]:
signature = np.full(shape=(N_HASHES, len(total_doc)), fill_value=len(shingle_set)+5)
for i in range(N_HASHES):
    for id in shingleID:
        hash_id = hash_funcs[i][id]
        for p in range(len(total_doc)):
            if shingle_set[shingleMap[id][1]][p]==1:
                if hash_id<signature[i][p]:
                    signature[i][p] = hash_id

In [11]:
def jaccard(c1,c2):
    """
    Calculates the Jaccard similarity of Columns C1 and C2
    """
    a=0
    b=0
    c=0
    for key in shingle_set.keys():
        if shingle_set[key][c1] == shingle_set[key][c2] and shingle_set[key][c1] == 1:
            a+=1
        elif shingle_set[key][c1] == 1:
            b+=1
        elif shingle_set[key][c2] == 1:
            c+=1
    return a/(a+b+c)

In [12]:
def hamming(c1, c2):
    """
    Calculates the Hamming distance between Columns C1 and C2
    """
    a=0
    for key in shingle_set.keys():
        if shingle_set[key][c1] != shingle_set[key][c2]:
            a+=1
    return a

In [13]:
def cosine(c1, c2):
    """
    Calculates the Cosine similarity of Columns C1 and C2
    """
    a=0
    b=0
    c=0
    for key in shingle_set.keys():
        if shingle_set[key][c1] == shingle_set[key][c2] and shingle_set[key][c1] == 1:
            a+=1
        if shingle_set[key][c1] == 1:
            b+=1
        if shingle_set[key][c2] == 1:
            c+=1
    return a/math.sqrt(b*c)

In [14]:
def euclidean(c1, c2):
    """
    Calculates the Euclidean Distance between Columns C1 and C2
    """
    a=0
    for key in shingle_set.keys():
        a+=(shingle_set[key][c1]-shingle_set[key][c2])**2    
    return math.sqrt(a)

In [15]:
rows = 2
bands = math.ceil(len(signature)/rows)
bucket_size = 3
buckets = np.zeros(shape=(bands, len(total_doc)))

In [16]:
for band in range(bands):
    for j in range(signature.shape[1]):
        b = 0
        for i in range(rows):
            b+=signature[band*rows + i][j]%bucket_size
        buckets[band][j]=b

In [17]:
doc1 = int(input("Enter a document ID "))
start = timeit.default_timer()
threshold = 2
candidates = []
for c in range(signature.shape[1]):
    count = 0
    for band in range(bands):
        if c !=doc1:
            if buckets[band][c] == buckets[band][doc1]:
                count+=1
    if count>=threshold:
        candidates.append(c)

print("The candidate pairs are-")
for c in candidates:
    print(str(doc1)+","+str(c))

jacc = [(jaccard(doc1, c), c) for c in candidates]
jacc.sort(reverse=True)
cos = [(cosine(doc1, c),c) for c in candidates]
cos.sort(reverse=True)
euc = [(euclidean(doc1, c),c) for c in candidates]
euc.sort()
ham = [(hamming(doc1, c),c) for c in candidates]
ham.sort()

print("\nJaccard Similarity")
for c in range(len(candidates)):
    print("Doc" + str(jacc[c][1]) +" "+ str(jacc[c][0]))

print("\nCosine Similarity")
for c in range(len(candidates)):
    print("Doc" + str(cos[c][1]) +" "+ str(cos[c][0]))

print("\nEuclidean Distance")
for c in range(len(candidates)):
    print("Doc" + str(euc[c][1]) +" "+ str(euc[c][0]))

print("\nHamming Distance")
for c in range(len(candidates)):
    print("Doc" + str(ham[c][1]) +" "+ str(ham[c][0]))


Enter a document ID 43
The candidate pairs are-
43,2
43,5
43,6
43,10
43,12
43,13
43,19
43,21
43,23
43,26
43,28
43,31
43,32
43,35
43,36
43,38
43,39
43,40
43,41
43,48
43,55
43,56
43,57
43,59
43,69
43,72
43,77
43,78
43,84
43,85
43,88
43,89
43,95
43,100
43,110
43,119
43,122
43,125
43,126
43,131
43,135
43,136
43,139
43,143
43,151
43,152
43,159
43,163
43,167
43,168
43,169
43,172
43,175
43,178
43,182
43,184
43,187
43,188
43,193
43,194
43,196
43,203
43,207
43,208
43,216
43,218
43,220
43,223
43,226
43,228
43,229
43,230
43,231
43,232
43,234
43,236
43,237
43,242
43,243
43,247
43,250
43,252
43,254
43,257
43,258
43,260
43,263
43,266
43,267
43,268
43,271
43,278
43,280
43,281
43,298
43,304
43,308
43,314
43,317
43,320
43,326
43,327
43,334
43,338
43,340
43,344
43,345
43,346
43,348
43,350
43,351
43,352
43,363
43,366
43,368
43,381
43,382
43,383
43,384
43,388
43,394
43,396

Jaccard Similarity
Doc40 0.05357142857142857
Doc85 0.047619047619047616
Doc317 0.038461538461538464
Doc234 0.018691588785046728
Doc25